# Mooring Load/Deflection Analysis for VolturnUS 15MW Floating Wind Turbine

In [ ]:
import glob
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
x = np.array([1,2,3])
np.repeat(x, 1)

## Get MoorDyn output data

In [ ]:
datadir = 'F:/Documents/WETO Incubator/OpenFAST/VolturnUS15MW' #change to directory containing MoorDyn output files
md_out_files = glob.glob(os.path.join(datadir, '*_500m.MD.Line*.out'))
numlines = len(md_out_files)
numnodes = 50

In [ ]:
out_dfs = []
for out_file in md_out_files:
    linenum = out_file[-5] # only valid for single-digit lines with filenames ending with '.out'
    out_df = pd.read_csv(out_file, sep='\s+', skiprows=[1], na_values=['***************'])
    out_df.columns = out_df.columns.str.replace('^Node', f'L{linenum}Node', regex=True)
    out_df.columns = out_df.columns.str.replace('^Seg', f'L{linenum}Seg', regex=True)
    out_dfs.append(out_df)

In [ ]:
mdout = pd.concat(out_dfs, axis=1)
mdout = mdout.loc[:, ~mdout.columns.duplicated()]
mdout = mdout.loc[mdout['Time']>90, :].reset_index()

## Functions

In [ ]:
def calc_max_scaling(basin_depth, basin_length, basin_width, full_depth, full_fp_length, full_fp_width):
    depth_scaling = full_depth / basin_depth
    fp_length_scaling = full_fp_length / basin_length
    fp_width_scaling = full_fp_width / basin_width
    return np.max([depth_scaling, fp_length_scaling, fp_width_scaling])

def scale_length(full_scale, scale_factor):
    return full_scale / scale_factor

def scale_force(full_scale, scale_factor):
    return full_scale / scale_factor ** 3

def scale_freq(full_scale, scale_factor):
    return full_scale / scale_factor ** -0.5

def get_power_spectrum(ts, dt):
    sp2 = np.fft.fft(ts, axis=0)
    sp = np.abs(sp2)[:len(sp2) // 2] * 2 # single-sided spectrum
    sp_rms_avg = np.sqrt(np.mean(sp**2, axis=1))[1:]
    freq = np.fft.fftfreq(len(ts), d=dt)[1:len(sp)]
    return sp_rms_avg, freq

def reshape_data(data, num_sets):
    return np.reshape(data, (-1, num_sets))

def get_dom_freqs(sp, freqs, tol):
    sp_norm = sp / np.sum(sp)
    return freqs[np.flatnonzero(sp_norm > tol)]

def freq_assessment(df, var, tol, num_sets):
    dt = df.loc[1, 'Time'] - df.loc[0, 'Time']
    datasets = reshape_data(df[var], num_sets)
    sp, freq = get_power_spectrum(datasets, dt)
    return get_dom_freqs(sp, freq, tol)

## Get maximum loads/deflections per mooring line

In [ ]:
maxs = {
    'Depth': np.zeros(numnodes+1),
    'Footprint Length': np.zeros(numnodes+1),
    'Footprint Width': np.zeros(numnodes+1),
}
for l in range(numlines):
   maxs[f'L{l+1} Max Hor Disp'] = np.zeros(numnodes+1)
   maxs[f'L{l+1} Max Vert Disp'] = np.zeros(numnodes+1)
   maxs[f'L{l+1} Max Disp Mag'] = np.zeros(numnodes+1)
   maxs[f'L{l+1} Max Tension'] = np.zeros(numnodes+1)
   maxs[f'L{l+1} Peak Freqs'] = []

for n in range(numnodes+1):
    max_depth = np.abs(mdout.loc[0, f'L1Node{n}pz'])
    fp_length = np.abs(mdout.loc[n, f'L1Node{n}px'] - mdout.loc[n, f'L2Node{n}px'])
    fp_width = np.abs(mdout.loc[n, f'L2Node{n}py'] - mdout.loc[n, f'L3Node{n}py'])
    for l in range(numlines):
      if n == 0:
        max_ten = np.nan
        peak_freqs = np.nan
      else:
        peak_freqs = freq_assessment(mdout, f'L{l+1}Seg{n}Ten', 0.05, 20)
        max_ten = mdout[f'L{l+1}Seg{n}Ten'].max()
      hor_disp = np.sqrt(
          (mdout[f'L{l+1}Node{n}px'] - mdout.loc[0, f'L{l+1}Node{n}px'])**2
        + (mdout[f'L{l+1}Node{n}py'] - mdout.loc[0, f'L{l+1}Node{n}py'])**2
      )
      vert_disp = mdout[f'L{l+1}Node{n}pz'] - mdout[f'L{l+1}Node{n}pz'][0]
      mag_disp = np.sqrt(
          (mdout[f'L{l+1}Node{n}px'] - mdout.loc[0, f'L{l+1}Node{n}px'])**2
        + (mdout[f'L{l+1}Node{n}py'] - mdout.loc[0, f'L{l+1}Node{n}py'])**2
        + (mdout[f'L{l+1}Node{n}pz'] - mdout.loc[0, f'L{l+1}Node{n}pz'])**2
      )
      max_hor_disp = hor_disp.max()
      max_vert_disp = vert_disp.max()
      max_mag_disp = mag_disp.max()

      # maxs['Partition Node'][n] = int(n)
      maxs[f'L{l+1} Max Disp Mag'][n] = max_mag_disp
      maxs[f'L{l+1} Max Hor Disp'][n] = max_hor_disp
      maxs[f'L{l+1} Max Vert Disp'][n] = max_vert_disp
      maxs[f'L{l+1} Max Tension'][n] = max_ten
      maxs[f'L{l+1} Peak Freqs'].append(peak_freqs)
    maxs['Depth'][n] = max_depth
    maxs['Footprint Length'][n] = fp_length
    maxs['Footprint Width'][n] = fp_width


### Save as `pandas` `DataFrame`

In [ ]:
node_maxs = pd.DataFrame(data=maxs)

In [ ]:
node_maxs

### Check time series tension

In [ ]:
plt.plot(mdout['Time'], mdout['L1Seg50Ten'], label='Line 1')
plt.plot(mdout['Time'], mdout['L2Seg50Ten'], label='Line 2')
plt.plot(mdout['Time'], mdout['L3Seg50Ten'], label='Line 3')

## Assess overall maxes at each node (aka partitioning point)

### UMaine basin bounds (to determine possible scaling)

In [ ]:
# UMaine basin dimensions (in meters)
w2_length = 30
w2_width = 9
w2_depth = 4.5

### Take highest value from all mooring lines of previous `DataFrame`

In [ ]:
partition_maxs = maxs = {
    'Max Hor Disp': np.zeros(numnodes),
    'Max Vert Disp': np.zeros(numnodes),
    'Max Disp Mag': np.zeros(numnodes),
    'Max Tension': np.zeros(numnodes),
    'Peak Freqs': [],
    'Physical Depth': np.zeros(numnodes),
    'Footprint Length': np.zeros(numnodes),
    'Footprint Width': np.zeros(numnodes),
    'Max Scaling': np.zeros(numnodes),
    'Scaled Hor Disp': np.zeros(numnodes),
    'Scaled Vert Disp': np.zeros(numnodes),
    'Scaled Disp Mag': np.zeros(numnodes),
    'Scaled Tension': np.zeros(numnodes),
    'Scaled Peak Freqs': []
}
partition_nodes = np.zeros(numnodes)

In [ ]:
for n in range(1, numnodes+1):
    max_phys_depth = node_maxs.loc[numnodes-n:, 'Depth'].max()
    max_fp_length = node_maxs.loc[numnodes-n:, 'Footprint Length'].max()
    max_fp_width = node_maxs.loc[numnodes-n:, 'Footprint Width'].max()
    max_hor_disp = node_maxs.filter(regex='Max Hor Disp$').loc[n, :].max()
    max_vert_disp = node_maxs.filter(regex='Max Vert Disp$').loc[n, :].max()
    max_mag_disp = node_maxs.filter(regex='Max Disp Mag$').loc[n, :].max()
    max_seg_ten = node_maxs.filter(regex='Max Tension$').loc[n, :].max()
    peak_freqs = np.unique(np.concatenate(
        node_maxs.filter(regex='Peak Freqs$').loc[n, :]))
    max_scaling = calc_max_scaling(w2_depth, w2_length, w2_width,
                                   max_phys_depth, max_fp_length, max_fp_width)
    scaled_hor_disp = scale_length(max_hor_disp, max_scaling)
    scaled_vert_disp = scale_length(max_vert_disp, max_scaling)
    scaled_mag_disp = scale_length(max_mag_disp, max_scaling)
    scaled_seg_ten = scale_force(max_seg_ten, max_scaling)
    scaled_peak_freqs = scale_freq(peak_freqs, max_scaling)

    partition_nodes[n-1] = int(n)
    partition_maxs['Max Disp Mag'][n-1] = max_mag_disp
    partition_maxs['Max Hor Disp'][n-1] = max_hor_disp
    partition_maxs['Max Vert Disp'][n-1] = max_vert_disp
    partition_maxs['Peak Freqs'].append(peak_freqs)
    partition_maxs['Max Tension'][n-1] = max_seg_ten
    partition_maxs['Physical Depth'][n-1] = max_phys_depth
    partition_maxs['Footprint Length'][n-1] = max_fp_length
    partition_maxs['Footprint Width'][n-1] = max_fp_width
    partition_maxs['Max Scaling'][n-1] = max_scaling
    partition_maxs['Scaled Hor Disp'][n-1] = scaled_hor_disp
    partition_maxs['Scaled Vert Disp'][n-1] = scaled_vert_disp
    partition_maxs['Scaled Disp Mag'][n-1] = scaled_mag_disp
    partition_maxs['Scaled Tension'][n-1] = scaled_seg_ten
    partition_maxs['Scaled Peak Freqs'].append(scaled_peak_freqs)
partition_maxs = pd.DataFrame(data=partition_maxs, index=partition_nodes)

In [ ]:
partition_maxs

### Save to CSV

In [ ]:
partition_maxs.to_csv('results_VolturnUS-15MW_500m_semitaut.csv')

## Scale everything to 1/60th scale

In [ ]:
scale = 60

In [ ]:
partition_60 = maxs = {
    'Max Hor Disp': np.zeros(numnodes),
    'Max Vert Disp': np.zeros(numnodes),
    'Max Disp Mag': np.zeros(numnodes),
    'Max Tension': np.zeros(numnodes),
    'Peak Freqs': [],
    'Physical Depth': np.zeros(numnodes),
    'Footprint Length': np.zeros(numnodes),
    'Footprint Width': np.zeros(numnodes),
    'Max Scaling': np.zeros(numnodes),
    'Scaled Hor Disp': np.zeros(numnodes),
    'Scaled Vert Disp': np.zeros(numnodes),
    'Scaled Disp Mag': np.zeros(numnodes),
    'Scaled Tension': np.zeros(numnodes),
    'Scaled Peak Freqs': []
}
partition_nodes = np.zeros(numnodes)

In [ ]:
peak_freqs

In [ ]:
for n in range(1, numnodes+1):
    max_phys_depth = node_maxs.loc[numnodes-n:, 'Depth'].max()
    max_fp_length = node_maxs.loc[numnodes-n:, 'Footprint Length'].max()
    max_fp_width = node_maxs.loc[numnodes-n:, 'Footprint Width'].max()
    max_hor_disp = node_maxs.filter(regex='Max Hor Disp$').loc[n, :].max()
    max_vert_disp = node_maxs.filter(regex='Max Vert Disp$').loc[n, :].max()
    max_mag_disp = node_maxs.filter(regex='Max Disp Mag$').loc[n, :].max()
    max_seg_ten = node_maxs.filter(regex='Max Tension$').loc[n, :].max()
    peak_freqs = np.unique(np.concatenate(
        node_maxs.filter(regex='Peak Freqs$').loc[n, :]))
    max_scaling = calc_max_scaling(w2_depth, w2_length, w2_width,
                                   max_phys_depth, max_fp_length, max_fp_width)

    hor_disp_60 = scale_length(max_hor_disp, scale)
    vert_disp_60 = scale_length(max_vert_disp, scale)
    mag_disp_60 = scale_length(max_mag_disp, scale)
    seg_ten_60 = scale_force(max_seg_ten, scale)
    peak_freqs_60 = scale_freq(peak_freqs, scale)

    partition_nodes[n-1] = int(n)
    partition_60['Max Disp Mag'][n-1] = max_mag_disp
    partition_60['Max Hor Disp'][n-1] = max_hor_disp
    partition_60['Max Vert Disp'][n-1] = max_vert_disp
    partition_60['Peak Freqs'].append(peak_freqs)
    partition_60['Max Tension'][n-1] = max_seg_ten
    partition_60['Physical Depth'][n-1] = max_phys_depth
    partition_60['Footprint Length'][n-1] = max_fp_length
    partition_60['Footprint Width'][n-1] = max_fp_width
    if max_scaling > scale:
        partition_60['Scaled Hor Disp'][n-1] = np.nan
        partition_60['Scaled Vert Disp'][n-1] = np.nan
        partition_60['Scaled Disp Mag'][n-1] = np.nan
        partition_60['Scaled Tension'][n-1] = np.nan
        partition_60['Scaled Peak Freqs'].append(np.nan)
    else:
        partition_60['Scaled Hor Disp'][n-1] = hor_disp_60
        partition_60['Scaled Vert Disp'][n-1] = vert_disp_60
        partition_60['Scaled Disp Mag'][n-1] = mag_disp_60
        partition_60['Scaled Tension'][n-1] = seg_ten_60
        partition_60['Scaled Peak Freqs'].append(peak_freqs_60)
partition_60 = pd.DataFrame(data=partition_60, index=partition_nodes)

In [ ]:
partition_60